# Asset Analysis
Developed by Steve Kuhlman

In [7]:
using WGLMakie, AlgebraOfGraphics,  XLSX, DataFrames, Dates
include("Common.jl")

readHoldings (generic function with 1 method)

In [3]:
#Note: Enter the folder containing data here
dataDir="../data"
marketDir=joinpath(dataDir, "market")
(markets, marketNames)=readMarkets(marketDir);

In [4]:
"Calculate the minumum closing value for each equity"
function getMinClose(gmarkets)
  closeHL=combine(gmarkets, :Close => minimum)
  minClose=Dict{String, Float64}()
  [minClose[closeHL[r, :Ticker]] = closeHL[r, :Close_minimum]  for r in 1:size(closeHL, 1)]
  minClose
end

"Get the first and last date for the charts"
function getDateRange(market)
  #TODO get actual first and last, don't assume positions
  first=market[1,:timestamp]
  last=market[end,:timestamp]
  dateRange=first : Day(1) : last
end

function plotTicker(markets, marketNames, pos)
  dateRange=getDateRange(markets[pos])
  lines!(days, markets[pos].Close, label=marketNames[pos])
end

plotTicker (generic function with 1 method)

In [5]:
dateRange=getDateRange(markets)
gmarkets=groupby(markets, :Ticker);
minClose=getMinClose(gmarkets)
normalize(x,y)= (x / minClose[y], minClose[y])
transform!(markets, [:Close, :Ticker] => ByRow(normalize) => [:CloseNorm, :minClose]);


In [ ]:
"Plot normalized values of investments"
plt=data(markets) * mapping(:timestamp, :CloseNorm, color=:Ticker) * visual(Lines)
draw(plt)

In [ ]:
"Plot market prices of investments"
#plt=data(markets) * mapping(:timestamp, :Close, color=:Ticker) * visual(Lines)
#draw(plt)

In [8]:
holdingsFile=joinpath(dataDir, "Accounts_HistoryFidelity1.xlsx")
#nameMap=(date=1, symbol=4, quantity=7, price=8)
nameMap=(1=>:date, 4=>:symbol, 7=>:quantity, 8=>:price)
rh=readHoldings(holdingsFile, nameMap, 3)

LoadError: ArgumentError: idxs[1] has type NTuple{4, Pair{Int64, Symbol}}; only Integer, Symbol, or string values allowed when indexing by vector